## Inception naive version

In [1]:
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D
from tensorflow.keras.layers import Add, Dense, BatchNormalization, Dropout
from tensorflow.keras.layers import ReLU, MaxPool2D, GlobalAvgPool2D, LeakyReLU
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential, save_model, load_model
import numpy as np
from tensorflow.keras.callbacks import TensorBoard
import time
import matplotlib.pyplot as plt
import datetime
import cv2
import os
import tensorflow as tf
import tensorflow.keras
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

from pathlib import Path
from tensorflow.keras.utils import to_categorical
from collections import Counter
from plotly.subplots import make_subplots
from tensorflow.keras.layers.experimental import preprocessing as ps
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pickle
import glob
print('tf version', tf.__version__)
print('keras version', tf.keras.__version__)
print('gpu is ','available' if tf.config.list_physical_devices('GPU') else 'not available')

tf version 2.2.0-rc1
keras version 2.2.4-tf
gpu is  not available


In [2]:
def inception_module(x, f1, f2, f3):
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(x)
    
    conv3 = Conv2D(f2, (3,3), padding='same', activation='relu')(x)
    
    conv5 = Conv2D(f3, (5,5), padding='same', activation='relu')(x)
    
    pool = MaxPool2D((3,3), strides=(1,1), padding='same')(x)
    
    out = concatenate([conv1, conv3, conv5, pool])
    return out

In [3]:
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1,1)):
    x = layers.Conv2D(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = layers.BatchNormalization(axis=3, scale=False)(x)
    x = layers.Activation('relu')(x)
    return x

In [4]:
img_input = Input(shape=(128,128,3))
classes = 49
Weight_path = ''
channel_axis = 3

In [10]:
x = conv2d_bn(img_input, 32, 3, 3, strides=(2,2), padding='valid')
x = conv2d_bn(x, 32, 3, 3, padding='valid')
x = conv2d_bn(x, 64, 3, 3)
x = layers.MaxPooling2D((3,3), strides=(2,2))(x)

x = conv2d_bn(x, 80, 1, 1, padding='valid')
x = conv2d_bn(x, 192, 3, 3, padding='valid')

x = layers.MaxPooling2D((3,3), strides=(2,2))(x)

x = inc_block_a(x)
x = inc_block_a(x)
x = inc_block_a(x)

x = reduction_block_a(x)

x = inc_block_b(x)
x = inc_block_b(x)
x = inc_block_b(x)
x = inc_block_b(x)

x = reduction_block_b(x)

x = inc_block_c(x)
x = inc_block_c(x)

x = layers.GlobalAveragePooling2D(name='avg_pool')(x)

x = layers.Dense(classes, activation='softmax', name='predictions')(x)

NameError: name 'layers' is not defined

In [5]:
def inc_block_a(x):
    branch1x1 = conv2d_bn(x, 64, 1, 1)
    
    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    
    branch_pool = AveragePooling2D((3,3), strides=(1,1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=channel_axis)
    return x

In [6]:
def reduction_block_a(x):
    branch3x3 = conv2d_bn(x, 384, 3, 3, strides=(2,2), padding='valid')
    
    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, strides=(2,2), padding='valid')
    
    branch_pool = layers.MaxPooling2D((3,3), strides=(2,2))(x)
    x = layers.concatenate([branch3x3, branch3x3dbl, branch_pool], axis=channel_axis)
    return x

In [7]:
def inc_block_b(x):
    branch1x1 = conv2d_bn(x, 192, 1, 1)
    
    branch7x7 = conv2d_bn(x, 128, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1)
    
    branch7x7dbl = conv2d_bn(x, 128, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 1, 7)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, 7, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 192, 1, 7)
    
    branch_pool = layers.AveragePooling2D((3,3), strides=(1,1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool], axis=channel_axis)
    return x

In [8]:
def reduction_block_b(x):
    branch3x3 = conv2d_bn(x, 192, 1, 1)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3, strides=(2,2), padding='valid')
    
    branch7x7x3 = conv2d_bn(x, 192, 1, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3, strides=(2,2), padding='valid')
    
    branch_pool = layers.MaxPooling2D((3,3), strides=(2,2))(x)
    x = layers.concatenate([branch3x3, branch7x7x3, branch_pool], axis=channel_axis)
    return x

In [9]:
def inc_block_c(x):
    branch1x1 = conv2d_bn(x, 320, 1, 1)
    
    branch3x3 = conv2d_bn(x, 384, 1, 1)
    branch3x3_1 = conv2d_bn(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn(branch3x3, 384, 3, 1)
    branch3x3 = layers.concatenate([branch3x3_1, branch3x3_2], axis=channel_axis)
    
    branch3x3dbl = conv2d_bn(x, 448, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 384, 3, 3)
    branch3x3dbl_1 = conv2d_bn(branch3x3dbl, 384, 1, 3)
    branch3x3dbl_2 = conv2d_bn(branch3x3dbl, 384, 3, 1)
    branch3x3dbl = layers.concatenate([branch3x3dbl_1, branch3x3dbl_2], axis=channel_axis)

    branch_pool = layers.AveragePooling2D((3,3), strides=(1,1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
    x = layers.concatenate([branch1x1, branch3x3, branch3x3dbl, branch_pool], axis=channel_axis)
    return x